### Red Agents

##### Matthew Jones

Scenario 1b has two rules-based Red Agents. The first is our good friend the B_lineAgent. This represents an actor who has inside information, so is able to beeline straight towards the OpServer.

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents import *
from CybORG.Shared.Actions import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'

env = CybORG(path,'sim')

agent = B_lineAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(16):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.184.240/28
DiscoverNetworkServices 10.0.184.252
ExploitRemoteService 10.0.184.252
PrivilegeEscalate User2
DiscoverNetworkServices 10.0.50.154
ExploitRemoteService 10.0.50.154
ExploitRemoteService 10.0.184.252
PrivilegeEscalate User2
DiscoverNetworkServices 10.0.50.154
ExploitRemoteService 10.0.50.154
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.50.144/28
DiscoverNetworkServices 10.0.50.147
ExploitRemoteService 10.0.50.147
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.153.178


This agent runs along a predetermined path to the Op_Server, but is smart enough able to recover its position if interrupted. We can see below after Blue Team restores some hosts, the agent works out where the error in and re-exploits its way to the Op_Server.

In [2]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
            
    print(action)
    print('Success:',obs['success'])

ExploitRemoteService 10.0.153.178
Success: FALSE
ExploitRemoteService 10.0.50.147
Success: TRUE
PrivilegeEscalate Enterprise2
Success: TRUE
DiscoverNetworkServices 10.0.153.178
Success: TRUE
ExploitRemoteService 10.0.153.178
Success: TRUE
PrivilegeEscalate Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE


The other red agent is the MeanderAgent. This performs a breadth first search on all known hosts, scanning each one in turn, before attempting a mix of exploit and privilege escalate on the rest. This is an extremely slow agent in contrast to the laser-focussed B_lineAgent.

In [3]:
agent = RedMeanderAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(46):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.212.208/28
DiscoverNetworkServices 10.0.212.216
DiscoverNetworkServices 10.0.212.211
DiscoverNetworkServices 10.0.212.222
DiscoverNetworkServices 10.0.212.217
DiscoverNetworkServices 10.0.212.220
PrivilegeEscalate User0
ExploitRemoteService 10.0.212.211
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.219.100
ExploitRemoteService 10.0.219.100
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.219.96/28
DiscoverNetworkServices 10.0.219.103
DiscoverNetworkServices 10.0.219.99
DiscoverNetworkServices 10.0.219.107
ExploitRemoteService 10.0.219.107
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.135.249
ExploitRemoteService 10.0.212.220
ExploitRemoteService 10.0.219.103
ExploitRemoteService 10.0.219.107
PrivilegeEscalate Enterprise2
ExploitRemoteService 10.0.219.100
PrivilegeEscalate Enterprise1
ExploitRemoteService 10.0.219.99
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.212.216
PrivilegeEscalate User3
ExploitRemoteService 10.0.135

The Meander Agent is also able to recover from Blue's disruption.

In [4]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(24):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(env.get_last_action('Red'))

Impact Op_Server0
DiscoverRemoteSystems 10.0.135.240/28
ExploitRemoteService 10.0.212.217
PrivilegeEscalate User4
ExploitRemoteService 10.0.219.103
ExploitRemoteService 10.0.219.107
PrivilegeEscalate Enterprise2
ExploitRemoteService 10.0.212.222
PrivilegeEscalate User2
ExploitRemoteService 10.0.135.249
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0


### Blue Agents

The BlueReactRemoveAgent will wait until it sees suspicious activity, before using remove on all the hosts it has flagged. However, due to the 5% change that Red's exploit is missed, Red will always eventually get to the Op_Server.

In [5]:
env = CybORG(path,'sim',agents={'Red':B_lineAgent})

agent = BlueReactRemoveAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Remove User2
Monitor
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor


The BlueReactRestoreAgent is the same as the React agent above, but uses the Restore action.

In [6]:
agent = BlueReactRestoreAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4


### Green Agent

An important part of CybORG Scenario1b is the Green agent, which represents the users on the network. The Green Agent is very simple, it only performs a scanning action on random hosts some of the time. This is only visible by Blue Agent.

In [7]:
agent = GreenAgent()

results = env.reset('Green')
obs = results.observation
action_space = results.action_space

for i in range(12):
    print(agent.get_action(obs,action_space))

GreenPortScan
Sleep
GreenPortScan
Sleep
Sleep
Sleep
Sleep
GreenPortScan
GreenPortScan
GreenPortScan
Sleep
GreenPortScan


### Keyboard Agent

The KeyboardAgent allows a human user to manually choose actions. This is useful for getting an intuition for the scenario.

In [8]:
from CybORG.Agents.Wrappers import RedTableWrapper

cyborg = CybORG(path, 'sim',agents={'Blue':BlueMonitorAgent})
env = RedTableWrapper(env=cyborg, output_mode='table')

agent = KeyboardAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(3):
    print(obs)
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.20.112/28 | 10.0.20.126 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

************************************ Turn 1: Observation *************************************

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.20.112/28 | 10.0.20.126 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

********************************* Turn 1: Command Selection **********************************

0 Sleep
1 DiscoverRemoteSystems
2 DiscoverNetworkServices
3 ExploitRemoteService
4 BlueKeep
5 EternalBlue
6 FTPDirectoryTraversal
7 Hara